In [1]:
import pandas as pd
import numpy as np

from transformers import pipeline, Conversation, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

from pathlib import Path

In [2]:
CLEAN_TRAIN_DATASET_PATH = Path("../data/WELFake_clean_train.csv")
RANDOM_SEED = 42

dataset_df = pd.read_csv(CLEAN_TRAIN_DATASET_PATH, index_col=0)
dataset_df

,title,text,label
45905,Exclusive: Foreign Isis Fighters Defend Mosul ...,\nForeign fighters for Isis are choosing to ...,1
37291,JUDGE JEANINE UNLOADS On Hillary: “How Did You...,You don t want to miss a second of Judge Jeani...,1
46730,Gunman attacks Saudi security forces at gate o...,RIYADH (Reuters) - Two Saudi guards were shot ...,0
66327,Indian Software Mogul: Hire Americans Now Beca...,A leading Indian software entrepreneur says In...,0
58329,Rep. Diaz-Balart: Liberals Against Trump Who F...,Florida Congressman Mario attacked the “doub...,0
...,...,...,...
37847,"To applause and boos, Kerry urges Congress to ...",CHICAGO (Reuters) - Failure to approve the Tra...,0
6384,TINGLE UP HIS LEG? NBC Paid Off Chris Matthews...,Here s yet another claim that s really iffy be...,1
55885,U.S. government shares technical details on No...,WASHINGTON (Reuters) - The U.S. government on ...,0
881,Trumps history of corruption is mind-boggling....,"In the heat of a presidential campaign, youd t...",0


In [3]:
train_ds = Dataset.from_pandas(dataset_df)
real_train_ds = train_ds.filter(lambda x: x["label"] == 1)

Filter:   0%|          | 0/56634 [00:00<?, ? examples/s]

In [4]:
MODEL_NAME = "gpt2"
RESPONSE_TEMPLATE = "### Article:"

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["title"])):
        text = f"### Title: {example['title'][i]} {RESPONSE_TEMPLATE} {example['text'][i]}"
        output_texts.append(text)
    return output_texts


model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

collator = DataCollatorForCompletionOnlyLM(RESPONSE_TEMPLATE, tokenizer=tokenizer)

In [5]:
trainer = SFTTrainer(
    model=model,
    train_dataset=real_train_ds,
    formatting_func=formatting_prompts_func,
    data_collator=collator
)

d:\mamba\envs\fake-news\lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/28514 [00:00<?, ? examples/s]

In [6]:
trainer.train()

  0%|          | 0/10695 [00:00<?, ?it/s]

d:\mamba\envs\fake-news\lib\site-packages\trl\trainer\utils.py:141: UserWarning: Could not find response key `### Article:` in the following instance: ### Title:  Trump ‘Fixes’ His Muslim Ban, Somehow Makes It EVEN WORSE (TWEETS) ### Article: If we ve learned anything from Donald Trump s constant flip-flopping on his policies and stances on issues, it s that whenever he tries to fix an idiotic statement he s made, he usually just manages to make it far worse. We saw this with his numerous stances on abortion, in which he first stated that women should be punished for terminating pregnancies, and then furiously backpedaled in several different directions once he realized that statement was too inflammatory.Trump is once again revising one of his policies, and this time it s a correction to one of the most controversial statements in his campaign   his proposed ban on Muslims. Having once said that all Muslims should be banned from coming into the United States, Trump has now changed it 

: 

In [5]:
conversation = Conversation([
    {
        "role": "system",
        "content": (
            "You are an news article generate. "
            "Your task is to generate real-sounding news article text based on a given article title."
        )
    },
    {
        "role": "user",
        "content": "President Biden collaborates with republicans on the crime bill."
    }
])

In [7]:
res_converation = chatbot(conversation)

TypeError: can only concatenate str (not "Conversation") to str